In [116]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [117]:
ln_trn=pd.read_csv('train_loan.csv')
ln_tst=pd.read_csv('test_loan.csv')
ln_ss=pd.read_csv('Sample_Submission_loan.csv')

In [118]:
ln_ss.head()

,Loan_ID,Loan_Status


In [119]:
ln_trn.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [120]:
ln_tst.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')

In [121]:
ln_trn.columns.difference(ln_tst.columns)

Index(['Loan_Status'], dtype='object')

In [122]:
print('Train shape:',ln_trn.shape)
print('Test Shape:',ln_tst.shape)

Train shape: (614, 13)
Test Shape: (367, 12)


In [123]:
ln_trn['Source']='Train'
ln_tst['Source']='Test'
ln_trn.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Source
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,Train
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,Train
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,Train
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,Train
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,Train


In [124]:
df_ln=pd.concat([ln_trn,ln_tst],axis=0,sort=False)
df_ln.isna().sum()

Loan_ID                0
Gender                24
Married                3
Dependents            25
Education              0
Self_Employed         55
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount            27
Loan_Amount_Term      20
Credit_History        79
Property_Area          0
Loan_Status          367
Source                 0
dtype: int64

In [125]:
df_ln.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 14 columns):
Loan_ID              981 non-null object
Gender               957 non-null object
Married              978 non-null object
Dependents           956 non-null object
Education            981 non-null object
Self_Employed        926 non-null object
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
LoanAmount           954 non-null float64
Loan_Amount_Term     961 non-null float64
Credit_History       902 non-null float64
Property_Area        981 non-null object
Loan_Status          614 non-null object
Source               981 non-null object
dtypes: float64(4), int64(1), object(9)
memory usage: 115.0+ KB


In [126]:
pp.ProfileReport(df_ln)

Number of variables,15
Number of observations,981
Total Missing (%),4.1%
Total size in memory,115.0 KiB
Average record size in memory,120.1 B
Numeric,6
Categorical,8
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [127]:
df_ln.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,981.000000,981.000000,954.000000,961.000000,902.000000
mean,5179.795107,1601.916330,142.511530,342.201873,0.835920
std,5695.104533,2718.772806,77.421743,65.100602,0.370553
min,0.000000,0.000000,9.000000,6.000000,0.000000
25%,2875.000000,0.000000,100.000000,360.000000,1.000000
50%,3800.000000,1110.000000,126.000000,360.000000,1.000000
75%,5516.000000,2365.000000,162.000000,360.000000,1.000000
max,81000.000000,41667.000000,700.000000,480.000000,1.000000


In [128]:
df_ln.nunique()

Loan_ID              981
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      752
CoapplicantIncome    437
LoanAmount           232
Loan_Amount_Term      12
Credit_History         2
Property_Area          3
Loan_Status            2
Source                 2
dtype: int64

In [129]:
def applymode(x):
    a=x.mode()[0]
    print(a)
    return x.fillna(a)

In [130]:
df_ln['Gender']=df_ln['Gender'].fillna(df_ln['Gender'].mode()[0])
#data['Native Country'] = data['Native Country'].fillna(data['Native Country'].mode()[0])

In [131]:
df_ln['Credit_History'].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [132]:
df_ln['Married']=df_ln['Married'].fillna(df_ln['Married'].mode()[0])

In [133]:
df_ln.isna().sum()

Loan_ID                0
Gender                 0
Married                0
Dependents            25
Education              0
Self_Employed         55
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount            27
Loan_Amount_Term      20
Credit_History        79
Property_Area          0
Loan_Status          367
Source                 0
dtype: int64

In [134]:
df_ln['Self_Employed']=df_ln['Self_Employed'].fillna(df_ln['Self_Employed'].mode()[0])

In [135]:
df_ln['Credit_History']=df_ln['Credit_History'].fillna(df_ln['Credit_History'].mode()[0])

In [136]:
df_ln['LoanAmount'] = df_ln['LoanAmount'].transform(lambda x : x.fillna(x.median()))

In [137]:
df_ln['Loan_Amount_Term'] = df_ln['Loan_Amount_Term'].transform(lambda x : x.fillna(x.median()))

In [138]:
df_ln['Loan_Status'] = df_ln['Loan_Status'].transform(lambda x : x.fillna('UNKNOWN'))

In [139]:
df_ln.nunique()

Loan_ID              981
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      752
CoapplicantIncome    437
LoanAmount           232
Loan_Amount_Term      12
Credit_History         2
Property_Area          3
Loan_Status            3
Source                 2
dtype: int64

In [140]:
df_ln_column_category = df_ln.select_dtypes(exclude=[np.number]).columns
df_ln_column_category

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status', 'Source'],
      dtype='object')

In [141]:
df_ln_column_Integer = df_ln.columns.difference(df_ln_column_category)
df_ln_column_Integer

Index(['ApplicantIncome', 'CoapplicantIncome', 'Credit_History', 'LoanAmount',
       'Loan_Amount_Term'],
      dtype='object')

In [142]:
df_ln_one_hot = pd.get_dummies(df_ln[df_ln_column_category].drop(columns = ["Loan_ID","Loan_Status"]))

In [143]:
df_ln.Loan_Status.value_counts()

Y          422
UNKNOWN    367
N          192
Name: Loan_Status, dtype: int64

In [144]:
values=df_ln['Loan_Status']
label_encoder=LabelEncoder()
integer_encoded_1=label_encoder.fit_transform(values)

In [145]:
df_ln=df_ln.drop(["Loan_Status"],axis=1)
df_ln["Loan_Status"] = integer_encoded_1
df_ln["Loan_Status"] = df_ln.Loan_Status.replace({1:3,2:1})
df_ln["Loan_Status"] = df_ln.Loan_Status.replace({3:2})
df_ln.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Source,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,126.0,360.0,1.0,Urban,Train,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,Train,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Train,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Train,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Train,1


In [146]:
df_ln_enco=df_ln.drop(['Gender', 'Married', 'Dependents', 'Education','Self_Employed', 'Property_Area', 
                       'Source'],axis=1)

In [147]:
df_ln_model = pd.concat([df_ln_enco,df_ln_one_hot],axis=1)
df_ln_model

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Source_Test,Source_Train
0,LP001002,5849,0.0,126.0,360.0,1.0,1,0,1,1,...,0,1,0,1,0,0,0,1,0,1
1,LP001003,4583,1508.0,128.0,360.0,1.0,0,0,1,0,...,0,1,0,1,0,1,0,0,0,1
2,LP001005,3000,0.0,66.0,360.0,1.0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,1
3,LP001006,2583,2358.0,120.0,360.0,1.0,1,0,1,0,...,0,0,1,1,0,0,0,1,0,1
4,LP001008,6000,0.0,141.0,360.0,1.0,1,0,1,1,...,0,1,0,1,0,0,0,1,0,1
5,LP001011,5417,4196.0,267.0,360.0,1.0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,1
6,LP001013,2333,1516.0,95.0,360.0,1.0,1,0,1,0,...,0,0,1,1,0,0,0,1,0,1
7,LP001014,3036,2504.0,158.0,360.0,0.0,0,0,1,0,...,1,1,0,1,0,0,1,0,0,1
8,LP001018,4006,1526.0,168.0,360.0,1.0,1,0,1,0,...,0,1,0,1,0,0,0,1,0,1
9,LP001020,12841,10968.0,349.0,360.0,1.0,0,0,1,0,...,0,1,0,1,0,0,1,0,0,1


In [148]:
df_ln_model.shape

(981, 24)

In [149]:
train_modified=df_ln_model.loc[df_ln_model.Source_Train==1,:]
test_modified=df_ln_model.loc[df_ln_model.Source_Train==0,:]
train_modified.shape

(614, 24)

In [150]:
train_modified.columns

Index(['Loan_ID', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'Gender_Female',
       'Gender_Male', 'Married_No', 'Married_Yes', 'Dependents_0',
       'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Source_Test', 'Source_Train'],
      dtype='object')

In [151]:
test_modified.columns

Index(['Loan_ID', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'Gender_Female',
       'Gender_Male', 'Married_No', 'Married_Yes', 'Dependents_0',
       'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Source_Test', 'Source_Train'],
      dtype='object')

In [152]:
test_modified.drop(columns=['Source_Test', 'Source_Train','Loan_Status'],inplace=True)
train_modified.drop(columns=['Source_Test', 'Source_Train'],inplace=True)
test_modified

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,LP001015,5720,0.0,110.0,360.0,1.0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,1
1,LP001022,3076,1500.0,126.0,360.0,1.0,0,1,0,1,...,1,0,0,1,0,1,0,0,0,1
2,LP001031,5000,1800.0,208.0,360.0,1.0,0,1,0,1,...,0,1,0,1,0,1,0,0,0,1
3,LP001035,2340,2546.0,100.0,360.0,1.0,0,1,0,1,...,0,1,0,1,0,1,0,0,0,1
4,LP001051,3276,0.0,78.0,360.0,1.0,0,1,1,0,...,0,0,0,0,1,1,0,0,0,1
5,LP001054,2165,3422.0,152.0,360.0,1.0,0,1,0,1,...,0,0,0,0,1,0,1,0,0,1
6,LP001055,2226,0.0,59.0,360.0,1.0,1,0,1,0,...,1,0,0,0,1,1,0,0,1,0
7,LP001056,3881,0.0,147.0,360.0,0.0,0,1,0,1,...,0,1,0,0,1,1,0,1,0,0
8,LP001059,13633,0.0,280.0,240.0,1.0,0,1,0,1,...,0,1,0,1,0,1,0,0,0,1
9,LP001067,2400,2400.0,123.0,360.0,1.0,0,1,1,0,...,0,0,0,0,1,1,0,0,1,0


In [155]:
train_modified.to_csv("train_ready_for_model_loan.csv",index=False)
test_modified.to_csv("test_ready_for_model_loan.csv",index=False)